In [1]:
import torch
%load_ext autoreload
%autoreload 2

In [2]:
from mamba_ssm import Mamba
from mamba_ssm import Mamba2

batch, length, dim = 2, 64, 16
x = torch.randn(batch, length, dim).to("cuda")

model = Mamba2(
    # This module uses roughly 3 * expand * d_model^2 parameters
    d_model=dim, # Model dimension d_model
    d_state=16,  # SSM state expansion factor
    d_conv=4,    # Local convolution width
    expand=2, # Block expansion factor
    headdim=1  
).to("cuda")

/home/ubuntu/miniconda3/envs/mamba2_env/lib/python3.8/site-packages/mamba_ssm/ops/selective_scan_interface.py:164: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  def forward(ctx, xz, conv1d_weight, conv1d_bias, x_proj_weight, delta_proj_weight,
/home/ubuntu/miniconda3/envs/mamba2_env/lib/python3.8/site-packages/mamba_ssm/ops/selective_scan_interface.py:240: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
  def backward(ctx, dout):
/home/ubuntu/miniconda3/envs/mamba2_env/lib/python3.8/site-packages/mamba_ssm/ops/triton/layer_norm.py:986: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  def forward(
/home/ubuntu/miniconda3/envs/mamba2_env/lib/python3.8/site-packages/mamba_ssm/ops/triton/layer_norm.py:1045: FutureWar

In [23]:
y = model(x)
assert y.shape == x.shape

NameError: name 'model' is not defined

# Caduceus Loading

In [24]:
from caduceus.modeling_caduceus import CaduceusForMaskedLM
from caduceus.configuration_caduceus import CaduceusConfig

In [28]:
config = CaduceusConfig(d_model=768, #Need to make sure that d_model * expand / headdim is a multiple of 8 otherwise inference will fail.
                        n_layer=4,
                        vocab_size=12,
                        complement_map={0: 0,1: 1,2: 2,3: 3,4: 4,5: 5,6: 6,7: 10,8: 9,9: 8,10: 7,11: 11 },
                        rms_norm=True,
                        fused_add_norm=True,
                        residual_in_fp32=True,
                        norm_epsilon=1e-5,
                        bidirectional=True,
                        bidirectional_strategy="add",
                        bidirectional_weight_tie=True,
                        rcps=True,
                        headdim=64)

In [29]:
caduceus = CaduceusForMaskedLM(config=config).to("cuda")

 Config getting passed to Mamba: {'layer_idx': 0, 'headdim': 4, 'device': None, 'dtype': None}
 Config getting passed to Mamba: {'layer_idx': 1, 'headdim': 4, 'device': None, 'dtype': None}
 Config getting passed to Mamba: {'layer_idx': 2, 'headdim': 4, 'device': None, 'dtype': None}
 Config getting passed to Mamba: {'layer_idx': 3, 'headdim': 4, 'device': None, 'dtype': None}


In [30]:
batch, length = 2, 64
x = torch.randint(low=0,high=11,size=(batch, length)).to("cuda")
y = caduceus(x)

0